In [36]:
# Author: Davide Aloi - PhD student - University of Birmingham

# Description: the script calculates median and max current density values for all three
# datasets (wp2a, wp1a and wp1b), for three ROIs: left M1, left Th and right CB. 
# generated using ROAST. The script then correlates measures derived from each subject's current density map
# (i.e. median and max values of the current density in M1 and Thalamus) with DCM measures (i.e. self connectivity in M1,
# thalamo-cortical connectivity, etc).

# Imports
import warnings
warnings.filterwarnings("ignore")
import os
import glob
import numpy as np
from nilearn import image, plotting
from nilearn.image import new_img_like
from scipy import ndimage
import scipy.io
import pandas as pd

In [9]:
## Parameters and variables: 
results_folder = 'D:\\roast-chapter3\\wp_all_results\\' # Folder with results
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' # Project folder

# Datasets names and subjects lists
#db_names = ['wp2a', 'wp1a', 'wp1b']
db_names = ['wp2a', 'wp1a', 'wp1b']

db_subjects = [['01','02','03','04','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','22','23','24'], # Wp2a
               ['03','04','05','07','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26'], # Wp1a
               ['01','02','03','04','05','06','07','08','09','10','11','12','13','15','16','17','18','19','21','22','23']] # Wp1b                              

## Loading AAL3 atlas and extracting M1 / Thalamus ROIs (regions of interest)
# AAL3 atlas paper: https://www.oxcns.org/papers/607%20Rolls%20Huang%20Lin%20Feng%20Joliot%202020%20AAL3.pdf 
AAl3_path = os.path.join(main_folder, 'rois', 'AAL3v1_1mm.nii')
AAL3_atlas = image.load_img(AAl3_path)

## Creating M1, Th and cerebellar masks from the AAL3 atlas. Load MNI template.
# AAL3 index for left M1 = 1
m1 = image.math_img("np.where(img == 1, 1, 0)", img = AAL3_atlas) 
# AAL3 index for TH = 121 - 149 (odd values only (left thalamus)) --> I'm not convinced about this one, ask Davinia
th = image.math_img("np.where(np.isin(img, np.arange(121, 150, 2)), 1, 0)", img = AAL3_atlas) 
# AAL3 index for right Cerebellum (cb) (102 and 108: cerebellar lobes IV-V and VIII)
cb = image.math_img("np.where(np.isin(img, np.array([102, 108])), 1, 0)", img = AAL3_atlas)

In [52]:
# Iterate all datasets
for db_id, db in enumerate(db_names):

    # Lists where current density values will be stored
    all_m1_medians = []
    all_th_medians = []
    all_cb_medians = []
    all_m1_max = []
    all_th_max = []
    all_cb_max = []

    # Loading results for each dataset (only current density maps)
    cd_maps = image.load_img(os.path.join(results_folder, db + '_all_cd_maps.nii'))
    #emag_maps = image.load_img(os.path.join(results_folder, str(db + '_all_emag_maps.nii')))
    #masks_maps = image.load_img(os.path.join(results_folder, str(db + '_all_masks_maps.nii')))
    print('Analysing dataset ' + db + ' - N. subjects: ' + str(cd_maps.shape[3]))
    
    for i in range(0, len(db_subjects[db_id])): # Iterate all subjects
        cd_map = image.index_img(cd_maps, i) # extracting volume of subject i 
        # which is equal to: cd_maps.get_fdata()[:,:,:,i], but returns a nibabel object

        ## Masking
        # Resampling masks to cd_maps shape (done only once)
        if not 'm1_resampled' in locals():
            m1_resampled = image.resample_to_img(m1, cd_maps, interpolation = 'nearest')
            th_resampled = image.resample_to_img(th, cd_maps, interpolation = 'nearest')
            cb_resampled = image.resample_to_img(cb, cd_maps, interpolation = 'nearest')

        cd_m1 = image.math_img("img * img2", img = cd_map, img2 = m1_resampled) # Applying m1 mask
        cd_th = image.math_img("img * img2", img = cd_map, img2 = th_resampled) # Applying th mask
        cd_cb = image.math_img("img * img2", img = cd_map, img2 = cb_resampled) # Applying cb mask

        # Assigning NaNs to values = 0, to exclude non M1, Th or CB voxels from the
        # calculation of the median and max values.
        cd_m1_nans = np.where(m1_resampled.get_fdata() == 0, np.nan, cd_m1.get_fdata())
        cd_th_nans = np.where(th_resampled.get_fdata() == 0, np.nan, cd_th.get_fdata())
        cd_cb_nans = np.where(cb_resampled.get_fdata() == 0, np.nan, cd_cb.get_fdata())

        ## Calculation of median and max current density values, within each ROI
        m1_cd_median = np.nanmedian(cd_m1_nans)
        th_cd_median = np.nanmedian(cd_th_nans)
        cb_cd_median = np.nanmedian(cd_cb_nans)
        m1_cd_max = np.nanmax(cd_m1_nans)
        th_cd_max = np.nanmax(cd_th_nans)
        cb_cd_max = np.nanmax(cd_cb_nans)

        # Saving means, medians and max values into their respective lists   
        all_m1_medians.append(m1_cd_median)
        all_th_medians.append(th_cd_median)  
        all_cb_medians.append(cb_cd_median) 
        all_m1_max.append(m1_cd_max)
        all_th_max.append(th_cd_max)   
        all_cb_max.append(cb_cd_max)

    # Saving .csv file with results
    dataframe = pd.DataFrame(list(zip(all_m1_medians, all_m1_max, all_th_medians,
                                all_th_max, all_cb_medians, all_cb_max)),
                                columns = ['m1_medians', 'm1_max', 'th_medians',
                                'th_max', 'cb_medians', 'cb_max'],
                                index = db_subjects[db_id])
    dataframe.to_csv(os.path.join(results_folder, db + '_current_density_results.csv'))
    print(dataframe.head()) # Just priting the first few results

Analysing dataset wp2a - N. subjects: 22
    m1_medians    m1_max  th_medians    th_max  cb_medians    cb_max
01    0.054124  1.208374    0.013650  0.106023    0.013781  0.115594
02    0.055064  1.807546    0.012737  0.050165    0.014684  0.143109
03    0.046193  2.001952    0.013467  0.067964    0.013622  0.090746
04    0.042422  0.738720    0.013865  0.056990    0.015775  0.090253
06    0.052187  1.311178    0.016848  0.056214    0.017543  0.159848
Analysing dataset wp1b - N. subjects: 21
    m1_medians    m1_max  th_medians    th_max  cb_medians    cb_max
01    0.026737  0.124895    0.022440  0.075784    0.082469  0.742356
02    0.026783  0.117095    0.025593  0.123986    0.081143  0.641692
03    0.025528  0.102260    0.020794  0.089887    0.068954  0.687909
04    0.023657  0.139041    0.021605  0.062512    0.068795  1.243865
05    0.022424  0.115248    0.021917  0.064007    0.063087  1.366402
